<p style="text-align: center;"><font size="8"><b>Defining Our Own Classes</b></font><br>


Hopefully by now you are all comfortable with functional programming, in other words defining and using functions. Python however is actually an object oriented programming language. So far we have avoided much discussion of what that actually means. Today's lecture mirrors chapter 6 in Goldwasser and Letscher.

Remember from way back how we defined objects and classes. An object in a single instance of a class. We have already several examples of classes. Remember backs to lists.

In [1]:
groceries = list()
type(groceries)

list

`groceries` is a *object* of type *list*. List is one of the classes built in to Python. A list has *attributes*, for example a length. It also has *methods*, for example `append`, or `pop`.

A NumPy array is another example of an object.

In [2]:
import numpy as np

a = np.array([1,2,3])
type(a)

numpy.ndarray

Tuples and dictionaries are also objects.

Today we will look at designing our own classes. Classes can be used for many tasks. All of the algorithms we have already seen, for example Newton's method, or LCG random number generators could be written as part of a class, although we wrote them as standalone functions.

This illustrates an important point. Python supports building classes, but unlike other object oriented languages (for example Java) it does not require them. You are likely proficient enough by now in Python to code up an algorithm without custom classes. So why do we need classes at all?

Classes are a nice way to group together a set of data and functions that operate on that data. This leads to modular code with more manageable (i.e. smaller) units. Even though you do not need classes (or functions for that mattter), they often provide a more elegant solution that is easier to extend late on.

In addition, outside the world of mathematical programming, classes can be a very natural way of thinking about problems. Most modern software development is based on classes and objects. If you decide later on to learn Java or C++ for example, knowledge of objects will be a big help.

# A Point Class

As our first example, we will create a simple `Point` class for representing a two dimensional point. We initialize the class by calling:

    class Point:
    
The keyword **class** tells Python that we are defining a new class. The identifier `Point` is the name of the class. This name must follow standard naming conventions: it can only contain letters, digits and underscores, it cannot start with a digit and it cannot be a reserved word. The colon marks the beginning of the block of code the serves as the body of the class. As is the case with all other blocks of code, indentation is needed to separate the body of the class from the rest of the code.

Within the body of the class we must define all the **methods** (a.k.a **member functions**) that will be supported by this class. The function definitions will be nested within the class body. The precise order of these functions is not important.



The first method we will look at is a special one named `__init__` (note *two* underscores before and after `init`). This is referred to as the **constructor**. Each time a caller creates a point, this method is automatically called by Python. Every class you define must have an `__init__` method. The primary purpose of the `__init__` method is to establish initial values for the **attributes** of the newly created object.

The `__init__` function for our Point class might look like:

In [3]:
class Point:
    def __init__(self):
        self._x = 0
        self._y = 0

We can noew create a Point object.

In [4]:
p = Point() # call the constructor, store in p

type(p) # check the type of p

__main__.Point

As was the case with the stand-alone functions that we covered earlier, member function declarations begin with the `def` keyword followed by the name of the method and any parameters followed by the body of the function. One difference between stand-alone functions and member functions is the use of the `self` parameter in the function signature. 

A user instantiates a point using the syntax `Point()`, without sending any explicit parameter. The *implicit* parameter `self` serves to internally identify the particular instance being created. Eventually the user may create several different points, each of which will have its own state stored in memory. 

The `self` identifier allows us to access members of this instance inside the class body using the standard syntax `object.membername`. For example in the `__init__` function we assign `self._x = 0` to establish an initial value for the attribute `_x`. This attributes becomes part of the object's internal representation. If instead we had called
  
    _x = 0
we would have created a local variable `_x` and set it to 0. This would have no effect on the internal state of our point.

The choice of attribute names is of course up to us. For this case `x` and `y` are natural choices, yet we intentionally choose `_x` and `_y`. This convention of prefixing underscores to member attributes is used to indicate that these attributes are for internal use only. Although it is possible to access the attributes outside the class, for reasons we'll discuss later we generally prefer to leave management of class attributes to the designers of the class.

Since we don't want the user accessing `_x` and `_y` directly, we will provide them with **accessors** and **mutators** to get and modify these values.

In [5]:
class Point:
    def __init__(self):
        self._x = 0
        self._y = 0
        
    def getX(self):
        return self._x
    
    def getY(self):
        return self._y
    
    def setX(self,value):
        self._x = value
        
    def setY(self, value):
        self._y = value

The function `getX` returns the value of the attribute `_x` to the caller, while the function `setX` sets the value of `_x` to a given value. Note that all accessors and mutators take as the first parameter the implict `self` parameter. The mutators then take an additional argument to set the value. 

A typical call might look like:

In [6]:
p1 = Point()
p1.setX(0.5)
p1.setY(1)

print("x =",p1.getX(), "y =",p1.getY())

x = 0.5 y = 1


Testing our class can be done in one of two ways. We can do exactly what we just did; that is defining a class and then testing the routines later in the same file or interpretive session. A better way to do test classes however is to save the class declaration in its own Python file. It can then by imported just like a module.

I have saved the class declaration for the Point class [here](https://raw.githubusercontent.com/lukasbystricky/ISC-3313/master/lectures/chapter6/code/SimplePoint.py). If you save it in the same directory as either your notebook or an interactive session you can call load and test the class like so:

In [7]:
from SimplePoint import Point

p1 = Point()
p1.setX(0.5)
p1.setY(1)

print("x =",p1.getX(), "y =",p1.getY())

x = 0.5 y = 1


## Completing the Point class


Let us know look at redesigning the Point class to support more interesting behaviour. 

To start with, suppose we didn't want the initial values of `_x` and `_y` to be 0. Suppose we want the user to be able to specify them. We can do this by passing in arguments to the constructor. Just like with functions, these arugments can be optional.

In [8]:
class Point:
    def __init__(self, x=0, y=0):
        # x and y are optional, the default value for both is 0
        self._x = x
        self._y = y
        
p1 = Point(5,2) # a point at (5,2)

We may also want to define a scaling function that scaling both coordinate values by a single factor.

In [9]:
def scale(self, factor):
    self._x *= factor
    self._y *= factor

We may also want to compute the distance between two points. Recall that the distance between two points $(x_1,y_1)$ and $(x_2,y_2)$ can be computed by the Pythagoras theorem:

$$ d = \sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}.$$

We can write a function that does just this.

In [10]:
def distance(self, other):
    dx = self._x - other._x
    dy = self._y - other._y
    
    return np.sqrt(dx**2 + dy**2)

Here `other` is another instance of the class `Point`. Note that we use the NumPy function `sqrt`, so we have to make sure that the NumPy module has been imported earlier.

Another possible useful function is a `normalize` function. This function scales our coordinates, so that the point has distance 1 from the origin. 

In [11]:
def normalize(self):
    mag = self.distance(Point()) # compute the distance from point to the origin
    if mag > 0:
        # if the magnitude is not 0, scale by 1 over magnitude
        self.scale(1/mag)

Note that this function does not take any additonal parameters, nor does it return anything. It simply modifies the `_x` and `_y` values.

Let's test out our completed Point class. You can download the class file [here](https://raw.githubusercontent.com/lukasbystricky/ISC-3313/master/lectures/chapter6/code/CompletedPoint.py).

In [28]:
from CompletedPoint import Point

p1 = Point(5,1)
p2 = Point(10,2)

print("Distance from p1 to p2 is", p1.distance(p2))
p1.normalize()
print("Distance from p1 to p2 is", p1.distance(p2))

Distance from p1 to p2 is 5.09901951359
Distance from p1 to p2 is 9.19803902719


## Doc Strings

As with functions, classes can and should have doc strings. In the files you downloaded I've included docstrings. Make sure to use these whenever possible.

In [13]:
help(Point.distance)

Help on function distance in module CompletedPoint:

distance(self, other)
    Computes the distance between this point and another point
    
    Parameters
    ----------
    other [Point] 
    
    Returns
    -------
    d [float] : distance between this point and other point



## Exercise

Use the [CompletedPoint.py](https://raw.githubusercontent.com/lukasbystricky/ISC-3313/master/lectures/chapter6/code/CompletedPoint.py) class to complete the following tasks.

1. Define the points (1,1), (10,2)
2. Determine the distance between the two points after they have been normalized

## Exercise

Create a class `Rectangle` that has the following attributes:
* width
* height

It should also support the following operations:
* accessors and mutators for width and height
* a function to scale the width and another one to scale the height
* a function to compute the area of the rectangle

# Special Methods

Some class methods have names starting and ending with a double underscore. We've already seen the `__init__` function for example. This constructor is called automatically when an instance is created. 

Other special methods allow us to perform arithmetic operations with other instances, for example "+", "-", "\*" etc. 

Other languages may call this operator overloading.

## Example, a Polynomial Class


Let's create a class that represents a polynomial. A polynomial is a function $p(x)$ of the form:
$$ p(x) = a_0 + a_1 x + a_2 x^2 + ... + a_n x^n.$$

Our polynomial class will take a list of coefficients $a_i$, $0\leq  i \leq n$. 

Let's start by defining the class and giving it a constructor.

In [14]:
class Polynomial:
    def __init__(self, coefficients):
        self._coeff = coefficients

We will also want a way to evaluate the Polynomial at a point $x$. 

In [22]:
class Polynomial:
    def __init__(self, coefficients):
        self._coeff = coefficients
        
    def evaluate(self, x):
        p = 0
        for i in range(len(self._coeff)):
            p += self._coeff[i]*x**i
            
        return p

We can test this out on the function $p(x) = 1 + 3x + 5x^2$.

In [23]:
p = Polynomial([1,3,5])
x = 0.5

print("p(",x,") =", p.evaluate(x))

p( 0.5 ) = 3.75


## Adding Objects

Let `a` and `b` be instances of some class `C`. Can we write `a + b`? Yes, if class `C` has a special method, `__add__`. 

    class C:
    ....
        __add__(self, other):
        .....
        
The `__add__` method should add the instances `self` and `other` (whatever that means for the class `C`) and return the result as an instance. 

When Python encounters `a+b` it will check if class `C` contains an `__add__` method and if it does interpret `a + b` as `a.__add__(b)`.

Specifically, say we have two polynomials, $p_1 = 1 + 3x + 5x^2$ and $p_2 = 4 + 8x - 9x^2 + 4x^3$. We can say that $p_1 + p_2 = 5 + 11x - 4x^2 + 4x^3$. 

How can we code this up?

In [24]:
class Polynomial:
    def __init__(self, coefficients):
        self._coeff = coefficients
        
    def evaluate(self, x):
        p = 0
        for i in range(len(self._coeff)):
            p += self._coeff[i]*x**i
            
        return p
    
    def __add__(self, other):
        # start with the longer list of coefficents and add the other list
        if len(self._coeff) > len(other._coeff):
            sum_coeff = self._coeff
            for i in range(len(other._coeff)):
                sum_coeff[i] += other._coeff[i]
                
        else:
            sum_coeff = other._coeff
            for i in range(len(self._coeff)):
                sum_coeff[i] += self._coeff[i]
                
        return Polynomial(sum_coeff)

In [25]:
p1 = Polynomial([1,3,5])
p2 = Polynomial([4,8,-9,4])

p3 = p1 + p2
x = 0.5
print("p3(",x,") =", p3.evaluate(x))

p3( 0.5 ) = 10.0


## Call Method

Computing the value of the mathematical function represented by the polynomial class is done by calling the function `p.evaluate(x)`. If we could call `p(x)` instead this would look more like an ordinary function. 

Such a syntax is possible using the `__call__` special method. 

In [26]:
class Polynomial:
    def __init__(self, coefficients):
        self._coeff = coefficients
        
    def __call__(self, x):
        s = 0
        for i in range(len(self._coeff)):
            s += self._coeff[i]*x**i
            
        return s
    
    def __add__(self, other):
        # start with the longer list of coefficents and add the other list
        if len(self._coeff) > len(other._coeff):
            sum_coeff = self._coeff
            for i in range(len(other._coeff)):
                sum_coeff[i] += other._coeff[i]
                
        else:
            sum_coeff = other._coeff
            for i in range(len(self._coeff)):
                sum_coeff[i] += self._coeff[i]
                
        return Polynomial(sum_coeff)

In [27]:
p1 = Polynomial([1,2,4])
x = 0.5
print("p(",x,") =", p1(x))

p( 0.5 ) = 3.0


Note that we no longer need an `evaluate` function. A good convention is to include a `__call__` method in all classes that represent a mathematical function.

Objects that include a `__call__` method are said to be *callable objects*. Plain functions are also called callable. 

The function `callable(a)` tests  whether `a` behaves as a callable, i.e. if `a` is a function or an object with a `__call__` method.

In [21]:
callable(p1)

True

## Other Special Methods

Given two objects `a` and `b` the standard arithmetic operators are defined by the following special methods:
* `a + b` : `a.__add__(b)`
* `a - b` : `a.__sub__(b)`
* `a * b` : `a.__mul__(b)`
* `a / b` : `a.__div__(b)`
* `a ** b` : `a.__pow__(b)`

In addition, there are other non arithmetic special methods:
* `len(a)` : `a.__len__()`
* `abs(a)` : `a.__abs__()`
* `a == b` : `a.__eq__(b)`
* `a > b` : `a.__gt__(b)`
* `a >= b` : `a.__ge__(b)`
* `a < b` : `a.__lt__(b)`
* `a <= b` : `a.__le__(b)`
* `a != b` : `a.__ne__(b)`
* `-a`     : `a.__neg__()`

There is also a boolean method `__bool__()` which returns true or false and allows us to call `if a` for example and the `__str__` method that returns a string representation of our object. This allows us to call `print` with our object as a parameter.

## Exercise

Add the following special methods to the `Point` class:
* `__add__`
* `__sub__`
* `__neg__`
* `__abs__` - returns the magnitude of the point (the distance from the origin)
* `__eq__` - checks if both the x and y coordinates of two points are equal
* `__str__` - returns a string representation of the point, i.e. "(x,y)"

Test your methods using the points (1,1) and (1.8,2).